# TensorFlow Text Generation x RNN
* TensorFlow Tutorial을 참고하여 Custom RNN 모델을 작성
* TF 1.15 적용
* 셰익스피어 데이터셋을 이용해 텍스트 생성 문제
    https://www.tensorflow.org/tutorials/text/text_generation?hl=ko

In [1]:
import tensorflow as tf
import numpy as np
import os
import time

## 1. 데이터 로드

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
# 문자열 디코딩
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# 텍스트 길이
print ('텍스트의 길이: {}자'.format(len(text)))

텍스트의 길이: 1115394자


In [4]:
# 텍스트의 처음 250자를 살펴봅니다
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# 파일의 고유 문자수를 출력합니다.
vocab = sorted(set(text))
print ('고유 문자수 {}개'.format(len(vocab)))

고유 문자수 65개


## 2. 텍스트 처리
    - 텍스트 벡터화 : 문자를 수치화해야 한다.
    - 두개의 lookup table을 만들어 문자=>숫자 맵핑, 숫자=>문자 맵핑

In [6]:
# 고유 문자(vocab)에서 인덱스로 맵핑
# {'a':0, 'b':1, ...}
char2idx = {u: i for i, u in enumerate(vocab)}
# ['a', 'b', ...]
idx2char = np.array(vocab)
# text => int(고유번호) 맵핑
text_as_int = np.array([char2idx[c] for c in text])

In [7]:
# 각 문자에 대한 정수 표현
print('{')
for char, _ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


## 3. 훈련 샘플과 타깃 생성 (tf.dataset)

In [8]:
# text를 샘플 시퀀스로 나눈다. seq_lenth 길이로
# 텍스트를 seq_length + 1 개의 청크로 나눈다.
# ex. input: "Hello", seq_length: 4 => 입력 시퀀스 "Hell", 타깃 시퀀스 "ello"

# 먼저 텍스트 벡터를 문자 인덱스의 스트림으로 변환
# seq_length: 입력에 대해 원하는 문장의 최대 길이
seq_length = 100
examples_per_epoch = len(text)//seq_length

# 훈련 샘플 / 타깃 생성
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# for i in char_dataset.take(4):
#     print(idx2char[i.numpy()])

In [65]:
print(text_as_int.shape)
print(examples_per_epoch)

(1115394,)
11153


In [9]:
iterator = char_dataset.make_initializable_iterator()
el = iterator.get_next()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.


In [10]:
with tf.Session() as sess:
    sess.run(iterator.initializer)
    for i in range(5):
        print(idx2char[sess.run(el)])

F
i
r
s
t


In [11]:
# batch 생성 (배치 수 : examples_per_epoch)
seq = char_dataset.batch(seq_length+1, drop_remainder=True)

In [66]:
print(seq)

<DatasetV1Adapter shapes: (101,), types: tf.int64>


In [67]:
iterator = seq.make_initializable_iterator()
el_seq = iterator.get_next()
with tf.Session() as sess:
    sess.run(iterator.initializer)
    for i in range(5):
        item = sess.run(el_seq)
        print(repr(''.join(idx2char[item])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [94]:
# 배치 개수 확인하기
iterator = seq.make_initializable_iterator()
el_seq = iterator.get_next()

n_batches_seq = len(text)//(seq_length+1)
print(n_batches_seq)

with tf.Session() as sess:
    sess.run(iterator.initializer)
    for i in range(n_batches_seq):
        item = sess.run(el_seq)
        print("alive! %d" % i)

alive! 0
alive! 1
alive! 2
alive! 3
alive! 4
alive! 5
alive! 6
alive! 7
alive! 8
alive! 9
alive! 10
alive! 11
alive! 12
alive! 13
alive! 14
alive! 15
alive! 16
alive! 17
alive! 18
alive! 19
alive! 20
alive! 21
alive! 22
alive! 23
alive! 24
alive! 25
alive! 26
alive! 27
alive! 28
alive! 29
alive! 30
alive! 31
alive! 32
alive! 33
alive! 34
alive! 35
alive! 36
alive! 37
alive! 38
alive! 39
alive! 40
alive! 41
alive! 42
alive! 43
alive! 44
alive! 45
alive! 46
alive! 47
alive! 48
alive! 49
alive! 50
alive! 51
alive! 52
alive! 53
alive! 54
alive! 55
alive! 56
alive! 57
alive! 58
alive! 59
alive! 60
alive! 61
alive! 62
alive! 63
alive! 64
alive! 65
alive! 66
alive! 67
alive! 68
alive! 69
alive! 70
alive! 71
alive! 72
alive! 73
alive! 74
alive! 75
alive! 76
alive! 77
alive! 78
alive! 79
alive! 80
alive! 81
alive! 82
alive! 83
alive! 84
alive! 85
alive! 86
alive! 87
alive! 88
alive! 89
alive! 90
alive! 91
alive! 92
alive! 93
alive! 94
alive! 95
alive! 96
alive! 97
alive! 98
alive! 99
alive! 100

alive! 1495
alive! 1496
alive! 1497
alive! 1498
alive! 1499
alive! 1500
alive! 1501
alive! 1502
alive! 1503
alive! 1504
alive! 1505
alive! 1506
alive! 1507
alive! 1508
alive! 1509
alive! 1510
alive! 1511
alive! 1512
alive! 1513
alive! 1514
alive! 1515
alive! 1516
alive! 1517
alive! 1518
alive! 1519
alive! 1520
alive! 1521
alive! 1522
alive! 1523
alive! 1524
alive! 1525
alive! 1526
alive! 1527
alive! 1528
alive! 1529
alive! 1530
alive! 1531
alive! 1532
alive! 1533
alive! 1534
alive! 1535
alive! 1536
alive! 1537
alive! 1538
alive! 1539
alive! 1540
alive! 1541
alive! 1542
alive! 1543
alive! 1544
alive! 1545
alive! 1546
alive! 1547
alive! 1548
alive! 1549
alive! 1550
alive! 1551
alive! 1552
alive! 1553
alive! 1554
alive! 1555
alive! 1556
alive! 1557
alive! 1558
alive! 1559
alive! 1560
alive! 1561
alive! 1562
alive! 1563
alive! 1564
alive! 1565
alive! 1566
alive! 1567
alive! 1568
alive! 1569
alive! 1570
alive! 1571
alive! 1572
alive! 1573
alive! 1574
alive! 1575
alive! 1576
alive! 1577
aliv

alive! 2302
alive! 2303
alive! 2304
alive! 2305
alive! 2306
alive! 2307
alive! 2308
alive! 2309
alive! 2310
alive! 2311
alive! 2312
alive! 2313
alive! 2314
alive! 2315
alive! 2316
alive! 2317
alive! 2318
alive! 2319
alive! 2320
alive! 2321
alive! 2322
alive! 2323
alive! 2324
alive! 2325
alive! 2326
alive! 2327
alive! 2328
alive! 2329
alive! 2330
alive! 2331
alive! 2332
alive! 2333
alive! 2334
alive! 2335
alive! 2336
alive! 2337
alive! 2338
alive! 2339
alive! 2340
alive! 2341
alive! 2342
alive! 2343
alive! 2344
alive! 2345
alive! 2346
alive! 2347
alive! 2348
alive! 2349
alive! 2350
alive! 2351
alive! 2352
alive! 2353
alive! 2354
alive! 2355
alive! 2356
alive! 2357
alive! 2358
alive! 2359
alive! 2360
alive! 2361
alive! 2362
alive! 2363
alive! 2364
alive! 2365
alive! 2366
alive! 2367
alive! 2368
alive! 2369
alive! 2370
alive! 2371
alive! 2372
alive! 2373
alive! 2374
alive! 2375
alive! 2376
alive! 2377
alive! 2378
alive! 2379
alive! 2380
alive! 2381
alive! 2382
alive! 2383
alive! 2384
aliv

alive! 3041
alive! 3042
alive! 3043
alive! 3044
alive! 3045
alive! 3046
alive! 3047
alive! 3048
alive! 3049
alive! 3050
alive! 3051
alive! 3052
alive! 3053
alive! 3054
alive! 3055
alive! 3056
alive! 3057
alive! 3058
alive! 3059
alive! 3060
alive! 3061
alive! 3062
alive! 3063
alive! 3064
alive! 3065
alive! 3066
alive! 3067
alive! 3068
alive! 3069
alive! 3070
alive! 3071
alive! 3072
alive! 3073
alive! 3074
alive! 3075
alive! 3076
alive! 3077
alive! 3078
alive! 3079
alive! 3080
alive! 3081
alive! 3082
alive! 3083
alive! 3084
alive! 3085
alive! 3086
alive! 3087
alive! 3088
alive! 3089
alive! 3090
alive! 3091
alive! 3092
alive! 3093
alive! 3094
alive! 3095
alive! 3096
alive! 3097
alive! 3098
alive! 3099
alive! 3100
alive! 3101
alive! 3102
alive! 3103
alive! 3104
alive! 3105
alive! 3106
alive! 3107
alive! 3108
alive! 3109
alive! 3110
alive! 3111
alive! 3112
alive! 3113
alive! 3114
alive! 3115
alive! 3116
alive! 3117
alive! 3118
alive! 3119
alive! 3120
alive! 3121
alive! 3122
alive! 3123
aliv

alive! 3817
alive! 3818
alive! 3819
alive! 3820
alive! 3821
alive! 3822
alive! 3823
alive! 3824
alive! 3825
alive! 3826
alive! 3827
alive! 3828
alive! 3829
alive! 3830
alive! 3831
alive! 3832
alive! 3833
alive! 3834
alive! 3835
alive! 3836
alive! 3837
alive! 3838
alive! 3839
alive! 3840
alive! 3841
alive! 3842
alive! 3843
alive! 3844
alive! 3845
alive! 3846
alive! 3847
alive! 3848
alive! 3849
alive! 3850
alive! 3851
alive! 3852
alive! 3853
alive! 3854
alive! 3855
alive! 3856
alive! 3857
alive! 3858
alive! 3859
alive! 3860
alive! 3861
alive! 3862
alive! 3863
alive! 3864
alive! 3865
alive! 3866
alive! 3867
alive! 3868
alive! 3869
alive! 3870
alive! 3871
alive! 3872
alive! 3873
alive! 3874
alive! 3875
alive! 3876
alive! 3877
alive! 3878
alive! 3879
alive! 3880
alive! 3881
alive! 3882
alive! 3883
alive! 3884
alive! 3885
alive! 3886
alive! 3887
alive! 3888
alive! 3889
alive! 3890
alive! 3891
alive! 3892
alive! 3893
alive! 3894
alive! 3895
alive! 3896
alive! 3897
alive! 3898
alive! 3899
aliv

...

alive! 7723
alive! 7724
alive! 7725
alive! 7726
alive! 7727
alive! 7728
alive! 7729
alive! 7730
alive! 7731
alive! 7732
alive! 7733
alive! 7734
alive! 7735
alive! 7736
alive! 7737
alive! 7738
alive! 7739
alive! 7740
alive! 7741
alive! 7742
alive! 7743
alive! 7744
alive! 7745
alive! 7746
alive! 7747
alive! 7748
alive! 7749
alive! 7750
alive! 7751
alive! 7752
alive! 7753
alive! 7754
alive! 7755
alive! 7756
alive! 7757
alive! 7758
alive! 7759
alive! 7760
alive! 7761
alive! 7762
alive! 7763
alive! 7764
alive! 7765
alive! 7766
alive! 7767
alive! 7768
alive! 7769
alive! 7770
alive! 7771
alive! 7772
alive! 7773
alive! 7774
alive! 7775
alive! 7776
alive! 7777
alive! 7778
alive! 7779
alive! 7780
alive! 7781
alive! 7782
alive! 7783
alive! 7784
alive! 7785
alive! 7786
alive! 7787
alive! 7788
alive! 7789
alive! 7790
alive! 7791
alive! 7792
alive! 7793
alive! 7794
alive! 7795
alive! 7796
alive! 7797
alive! 7798
alive! 7799
alive! 7800
alive! 7801
alive! 7802
alive! 7803
alive! 7804
alive! 7805
aliv

alive! 8505
alive! 8506
alive! 8507
alive! 8508
alive! 8509
alive! 8510
alive! 8511
alive! 8512
alive! 8513
alive! 8514
alive! 8515
alive! 8516
alive! 8517
alive! 8518
alive! 8519
alive! 8520
alive! 8521
alive! 8522
alive! 8523
alive! 8524
alive! 8525
alive! 8526
alive! 8527
alive! 8528
alive! 8529
alive! 8530
alive! 8531
alive! 8532
alive! 8533
alive! 8534
alive! 8535
alive! 8536
alive! 8537
alive! 8538
alive! 8539
alive! 8540
alive! 8541
alive! 8542
alive! 8543
alive! 8544
alive! 8545
alive! 8546
alive! 8547
alive! 8548
alive! 8549
alive! 8550
alive! 8551
alive! 8552
alive! 8553
alive! 8554
alive! 8555
alive! 8556
alive! 8557
alive! 8558
alive! 8559
alive! 8560
alive! 8561
alive! 8562
alive! 8563
alive! 8564
alive! 8565
alive! 8566
alive! 8567
alive! 8568
alive! 8569
alive! 8570
alive! 8571
alive! 8572
alive! 8573
alive! 8574
alive! 8575
alive! 8576
alive! 8577
alive! 8578
alive! 8579
alive! 8580
alive! 8581
alive! 8582
alive! 8583
alive! 8584
alive! 8585
alive! 8586
alive! 8587
aliv

alive! 9302
alive! 9303
alive! 9304
alive! 9305
alive! 9306
alive! 9307
alive! 9308
alive! 9309
alive! 9310
alive! 9311
alive! 9312
alive! 9313
alive! 9314
alive! 9315
alive! 9316
alive! 9317
alive! 9318
alive! 9319
alive! 9320
alive! 9321
alive! 9322
alive! 9323
alive! 9324
alive! 9325
alive! 9326
alive! 9327
alive! 9328
alive! 9329
alive! 9330
alive! 9331
alive! 9332
alive! 9333
alive! 9334
alive! 9335
alive! 9336
alive! 9337
alive! 9338
alive! 9339
alive! 9340
alive! 9341
alive! 9342
alive! 9343
alive! 9344
alive! 9345
alive! 9346
alive! 9347
alive! 9348
alive! 9349
alive! 9350
alive! 9351
alive! 9352
alive! 9353
alive! 9354
alive! 9355
alive! 9356
alive! 9357
alive! 9358
alive! 9359
alive! 9360
alive! 9361
alive! 9362
alive! 9363
alive! 9364
alive! 9365
alive! 9366
alive! 9367
alive! 9368
alive! 9369
alive! 9370
alive! 9371
alive! 9372
alive! 9373
alive! 9374
alive! 9375
alive! 9376
alive! 9377
alive! 9378
alive! 9379
alive! 9380
alive! 9381
alive! 9382
alive! 9383
alive! 9384
aliv

alive! 10051
alive! 10052
alive! 10053
alive! 10054
alive! 10055
alive! 10056
alive! 10057
alive! 10058
alive! 10059
alive! 10060
alive! 10061
alive! 10062
alive! 10063
alive! 10064
alive! 10065
alive! 10066
alive! 10067
alive! 10068
alive! 10069
alive! 10070
alive! 10071
alive! 10072
alive! 10073
alive! 10074
alive! 10075
alive! 10076
alive! 10077
alive! 10078
alive! 10079
alive! 10080
alive! 10081
alive! 10082
alive! 10083
alive! 10084
alive! 10085
alive! 10086
alive! 10087
alive! 10088
alive! 10089
alive! 10090
alive! 10091
alive! 10092
alive! 10093
alive! 10094
alive! 10095
alive! 10096
alive! 10097
alive! 10098
alive! 10099
alive! 10100
alive! 10101
alive! 10102
alive! 10103
alive! 10104
alive! 10105
alive! 10106
alive! 10107
alive! 10108
alive! 10109
alive! 10110
alive! 10111
alive! 10112
alive! 10113
alive! 10114
alive! 10115
alive! 10116
alive! 10117
alive! 10118
alive! 10119
alive! 10120
alive! 10121
alive! 10122
alive! 10123
alive! 10124
alive! 10125
alive! 10126
alive! 10127

alive! 10823
alive! 10824
alive! 10825
alive! 10826
alive! 10827
alive! 10828
alive! 10829
alive! 10830
alive! 10831
alive! 10832
alive! 10833
alive! 10834
alive! 10835
alive! 10836
alive! 10837
alive! 10838
alive! 10839
alive! 10840
alive! 10841
alive! 10842
alive! 10843
alive! 10844
alive! 10845
alive! 10846
alive! 10847
alive! 10848
alive! 10849
alive! 10850
alive! 10851
alive! 10852
alive! 10853
alive! 10854
alive! 10855
alive! 10856
alive! 10857
alive! 10858
alive! 10859
alive! 10860
alive! 10861
alive! 10862
alive! 10863
alive! 10864
alive! 10865
alive! 10866
alive! 10867
alive! 10868
alive! 10869
alive! 10870
alive! 10871
alive! 10872
alive! 10873
alive! 10874
alive! 10875
alive! 10876
alive! 10877
alive! 10878
alive! 10879
alive! 10880
alive! 10881
alive! 10882
alive! 10883
alive! 10884
alive! 10885
alive! 10886
alive! 10887
alive! 10888
alive! 10889
alive! 10890
alive! 10891
alive! 10892
alive! 10893
alive! 10894
alive! 10895
alive! 10896
alive! 10897
alive! 10898
alive! 10899

In [71]:
# chunk 생성
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = seq.map(split_input_target)

In [72]:
dataset

<DatasetV1Adapter shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [50]:
iterator = dataset.make_initializable_iterator()
get_next = iterator.get_next()
with tf.Session() as sess:
    sess.run(iterator.initializer)
    input_ex, target_ex = sess.run(get_next)
    print(repr(''.join(idx2char[input_ex])))
    print(repr(''.join(idx2char[target_ex])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [52]:
get_next

(<tf.Tensor 'IteratorGetNext_13:0' shape=(100,) dtype=int64>,
 <tf.Tensor 'IteratorGetNext_13:1' shape=(100,) dtype=int64>)

### Time Step
각 char에 매치되는 idx는 하나의 time step  
time step 0의 입력으로 모델은 F의 idx 를 받고 다음 문자로 i의 idx를 예측  
RNN은 현재 입력 문자 F 외에도 이전 time step의 context를 고려

In [15]:
for i, (input_idx, target_idx) in enumerate(zip(input_ex[:5], target_ex[:5])):
    print("{:4d}단계".format(i))
    print("  입력: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  예상 출력: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

   0단계
  입력: 18 ('F')
  예상 출력: 47 ('i')
   1단계
  입력: 47 ('i')
  예상 출력: 56 ('r')
   2단계
  입력: 56 ('r')
  예상 출력: 57 ('s')
   3단계
  입력: 57 ('s')
  예상 출력: 58 ('t')
   4단계
  입력: 58 ('t')
  예상 출력: 1 (' ')


훈련 배치 생성

In [73]:
BATCH_SIZE = 64
# 데이터셋을 섞을 버퍼의 크기 (전체 시퀀스를 메모리에서 섞지 않음)
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [74]:
dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [95]:
# 배치 개수 확인하기
iterator = dataset.make_initializable_iterator()
get_next = iterator.get_next()

n_batches_dataset = n_batches_seq//64
print(n_batches_dataset)

with tf.Session() as sess:
    sess.run(iterator.initializer)
    for i in range(n_batches_dataset):
        item = sess.run(get_next)
        print("alive! %d" % i)

172
alive! 0
alive! 1
alive! 2
alive! 3
alive! 4
alive! 5
alive! 6
alive! 7
alive! 8
alive! 9
alive! 10
alive! 11
alive! 12
alive! 13
alive! 14
alive! 15
alive! 16
alive! 17
alive! 18
alive! 19
alive! 20
alive! 21
alive! 22
alive! 23
alive! 24
alive! 25
alive! 26
alive! 27
alive! 28
alive! 29
alive! 30
alive! 31
alive! 32
alive! 33
alive! 34
alive! 35
alive! 36
alive! 37
alive! 38
alive! 39
alive! 40
alive! 41
alive! 42
alive! 43
alive! 44
alive! 45
alive! 46
alive! 47
alive! 48
alive! 49
alive! 50
alive! 51
alive! 52
alive! 53
alive! 54
alive! 55
alive! 56
alive! 57
alive! 58
alive! 59
alive! 60
alive! 61
alive! 62
alive! 63
alive! 64
alive! 65
alive! 66
alive! 67
alive! 68
alive! 69
alive! 70
alive! 71
alive! 72
alive! 73
alive! 74
alive! 75
alive! 76
alive! 77
alive! 78
alive! 79
alive! 80
alive! 81
alive! 82
alive! 83
alive! 84
alive! 85
alive! 86
alive! 87
alive! 88
alive! 89
alive! 90
alive! 91
alive! 92
alive! 93
alive! 94
alive! 95
alive! 96
alive! 97
alive! 98
alive! 99
alive!

## 4. 모델 설계
![img.png](https://tensorflow.org/tutorials/text/images/text_generation_training.png)

In [18]:
# hyperparameters
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

## A. 케라스를 이용한 모델

### Build Model

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
model = build_model(vocab_size=len(vocab),
                  embedding_dim=embedding_dim,
                  rnn_units=rnn_units,
                  batch_size=BATCH_SIZE)

In [ ]:
# run model
iterator_tf = dataset.make_initializable_iterator()
get_next_tf = iterator_tf.get_next()

with tf.Session() as sess:
    sess.run(iterator_tf.initializer)
    input_ex, target_ex = sess.run(get_next_tf)
    
    example_batch_predictions = model(input_ex)
    print(example_batch_predictions.shape, "# (배치 크기, 시퀀스 길이, 어휘 사전 크기)")

In [ ]:
model.summary()

In [ ]:
example_batch_predictions

In [ ]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
    sampled_indices = tf.squeeze(sampled_indices,axis=-1)
    sampled_indices = sampled_indices.eval()

In [ ]:
sampled_indices

In [ ]:
print("입력: \n", repr("".join(idx2char[input_ex[0]])))
print()
print("예측된 다음 문자: \n", repr("".join(idx2char[sampled_indices])))

### Optimizer & Loss function
이 모델은 로짓을 반환하기 때문에 from_logits 플래그를 설정해야 합니다.

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    example_batch_loss  = loss(target_ex, example_batch_predictions)
    print("예측 배열 크기(shape): ", example_batch_predictions.shape, " # (배치 크기, 시퀀스 길이, 어휘 사전 크기")
    print("스칼라 손실:          ", example_batch_loss.eval().mean())

In [ ]:
model.compile('adam', loss=loss)
# model.compile('rmsprop', 'mse', target_tensors=[iterator.get_next()])

### Training

In [ ]:
EPOCHS=10
# 체크포인트가 저장될 디렉토리
# checkpoint_dir = './training_checkpoints'
# # 체크포인트 파일 이름
# checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

# checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_prefix,
#     save_weights_only=True)

history = model.fit(dataset, epochs=EPOCHS)  #, callbacks=[checkpoint_callback])

In [ ]:
# state = np.zeros((sample_x.shape[0], HIDDEN_SIZE))
    
# _total_loss, _ = sess.run([total_loss, train],
#                                feed_dict={rnn_input: sample_x,
#                                           rnn_output: sample_y,
#                                           initial_state: state})
# print(f'[Epoch:{epoch+1}] loss: {_total_loss:<7.4}', end='\r')

## B. NumPy를 이용한 모델
- Reference
    - https://victorzhou.com/blog/intro-to-rnns/
    - https://ratsgo.github.io/natural%20language%20processing/2017/03/09/rnnlstm/

### 4.1 Embedding

In [96]:
def char2tensor(char):
    tensor = np.zeros((vocab_size, 1))
    tensor[char2idx[char]][0] = 1
    return tensor

def line2tensor(line):
    tensor = np.zeros((len(line), vocab_size, 1))
    for li, letter in enumerate(line):
        tensor[li][char2idx[char]][0] = 1
    return tensor

def idx2tensor(idx):
    tensor = np.zeros((vocab_size, 1))
    tensor[idx][0] = 1
    return tensor

def idxs2tensor(idxs):
    tensor = np.zeros((len(idxs), vocab_size, 1))
    for li, letter in enumerate(idxs):
        tensor[li][letter][0] = 1
    return tensor

In [97]:
# a = [0, 1, 0, 0, ...]
char2tensor('a')

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [98]:
iterator_dataset = dataset.make_one_shot_iterator()        # 한번에 배치 만들어줌
get_next_dataset = iterator_dataset.get_next()

with tf.Session() as sess:
    input_ex_batch, target_ex_batch = sess.run(get_next_dataset)
            
    # BATCH_SIZE 만큼
    for i, (input_ex, target_ex) in enumerate(zip(input_ex_batch, target_ex_batch)):
        # tensor는 한 문장 (100, 65, 1)
        input_tensor = idxs2tensor(input_ex)

In [81]:
el_dataset

(<tf.Tensor 'IteratorGetNext_23:0' shape=(64, 100) dtype=int64>,
 <tf.Tensor 'IteratorGetNext_23:1' shape=(64, 100) dtype=int64>)

In [82]:
input_ex_batch

array([[58, 53, 50, ..., 63,  1, 57],
       [58, 50, 43, ...,  1, 46, 47],
       [39, 51, 47, ...,  1, 61, 43],
       ...,
       [27, 10,  0, ..., 24, 13, 10],
       [58, 46, 43, ..., 56,  1, 59],
       [53, 56,  1, ..., 58,  1, 49]])

In [83]:
input_tensor = idxs2tensor(input_ex)
print(input_tensor.shape)
print(input_tensor)

(100, 65, 1)
[[[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [1.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 ...

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [1.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]]


In [35]:
np.where(input_tensor==1)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
 array([39,  1, 58, 47, 52, 49, 43, 56,  5, 57,  0, 61, 47, 44, 43,  1, 61,
        47, 58, 46, 47, 52,  1, 39,  1, 51, 47, 50, 43,  1, 61, 46, 43, 56,
        43,  1, 51, 63,  1, 50, 39, 52, 42,  1, 39, 52, 42,  1, 50, 47, 60,
        47, 52, 45,  1, 50, 47, 43, 57, 11,  0, 39, 52, 42,  6,  1, 46, 39,
        60, 47, 52, 45,  1, 44, 50, 53, 61, 52,  1, 53, 60, 43, 56,  1, 51,
        39, 52, 63,  1, 49, 52, 39, 60, 47, 57, 46,  1, 54, 56, 53]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 

### 4.2 Simple RNN Cell
- Many-to-many
- Tx = Ty (fixed-length)
![image](https://stanford.edu/~shervine/teaching/cs-230/illustrations/rnn-many-to-many-same-ltr.png?2790431b32050b34b80011afead1f232)

In [211]:
from src.layers import tanh, softmax

class RNNCell:    
    def __init__(self
                , input_dim=None
                , output_dim=None
                , hidden_size=32
                , initial_state=None
                , activation=tanh
                , seed=1
                ):
        '''
        ================ parameters =================
        input  : inputs
        shape  : (sequence_length, embedding_size, 1)
        =============================================
        output : y 
        shape  : (inputs.shape[0]  # sequence_length
                , output_dim       # embedding_size
                , 1)
        '''

        # init
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_size = hidden_size
        
        if output_dim is None:
            self.output_dim=hidden_size
        if initial_state is None:
            self.initial_state = np.zeros((hidden_size, 1))
        else:
            self.initial_state = np.reshape(initial_state, (hidden_size, 1))

        np.random.seed(seed)
        # weights
        self.Wxh = np.random.normal(size=(hidden_size, input_dim)).round(3)
        self.Whh = np.random.normal(size=(hidden_size, hidden_size)).round(3)
        self.Why = np.random.normal(size=(output_dim,  hidden_size)).round(3)
        # bias
        self.bh = np.zeros((hidden_size, 1)).round(3)
        self.by = np.zeros((output_dim,  1)).round(3)
        self.activation = activation
        
        # back prop
        self.last_inputs = None
        self.last_hs = {0: initial_state}
        
    def forward(self, inputs):
        # tanh를 적용하는 순간 ht가 1로 수렴한다. (30 넘는 값)
        # word embedding이 필요한듯
        # 초기화 문제? yes ... normal(음수 필요), glorot_uniform(?)
    
        ht = self.initial_state # hidden state
        outputs = np.zeros((inputs.shape[0], self.output_dim, 1)) # (100, 65, 1)
        states = np.zeros((inputs.shape[0], self.hidden_size, 1))
        
        # back prop
        self.last_inputs = inputs
        self.last_hs = {0: ht}
        
        for i, xt in enumerate(inputs): # 모든 글자에 대해 (input: 'hello', xt: 'h')
            xt = np.reshape(xt, (self.input_dim, 1))
            # xt 가 1 x input_dim 이면?
            # hidden state
            ht = self.activation(self.Wxh @ xt + self.Whh @ ht + self.bh)
            self.last_hs[i+1] = ht
            # predicted output y
            y = (self.Why @ ht) + self.by
            y = softmax(y)
            outputs[i] = y[:,]
            states[i] = ht
        
        return outputs, states  # 100개의 y, ht
    
    def backprop(self, dy, learning_rate=1e-3):
        time = len(self.last_inputs)
        
        # calculate dL/dWhy & dL/dby
        # (output_dim x 1) * (1 x hidden_size) = (output_dim x hidden_size)
        dWhy = dy @ self.last_hs[time].T
        dby = dy
        # init
        dWhh = np.zeros_like(self.Whh)
        dWxh = np.zeros_like(self.Wxh)
        dbh = np.zeros_like(self.bh)
        # (hidden_size x 1) = (hidden_size x output_dim) x (output_dim x 1)
        dh = self.Why.T @ dy
        
        for t in reversed(range(time)):
            dh_raw = None
            if self.activation == tanh:
                dh_raw = (1 - self.last_hs[t+1]**2)
            elif self.activation == sigmoid:
                dh_raw = (1 - self.last_hs[t+1]) * self.last_hs[t+1]
            else: 
                return
            dh_raw *= dh
            
            dbh += dh_raw
            # (hs x hs) = (hs x 1) x (1 x hs)
            dWhh += dh_raw @ self.last_hs[t].T
            # (hs x input_dim) = (hs x 1) x (1 x input_dim)
            dWxh += dh_raw @ self.last_inputs[t].T
            dh = self.Whh @ dh_raw  # ??
        
        # Update weights and biases using GD
        self.Whh -= learning_rate * dWhh
        self.Wxh -= learning_rate * dWxh
        self.Why -= learning_rate * dWhy
        self.bh -= learning_rate * dbh
        self.by -= learning_rate * dby

In [192]:
# 문장 하나에 대한 테스트 (100, 65, 1)
output_dim = 65
hidden_size = 128
hidden_state = np.zeros((hidden_size, ))

rnn = RNNCell(input_dim=vocab_size  # feature 수
           , output_dim=output_dim
           , hidden_size=hidden_size
           , initial_state=hidden_state)

output, hidden_state = rnn.forward(input_tensor)

print("======== output ========")
print("shape: %s" % str(output.shape))
print([np.argmax(item, axis=0)[0] for item in output])

(128, 65)
(65, 1)
(128, 1)
(128, 128)
(128, 1)
(128, 1)
(128, 1)
[[ 0.70891534]
 [-0.95249369]
 [-0.0728706 ]
 [ 0.35048801]
 [-0.70035656]
 [ 0.54973398]
 [ 0.89726346]
 [ 0.5999058 ]
 [ 0.80085794]
 [-0.77749287]
 [-0.90240055]
 [ 0.77510875]
 [ 0.62061209]
 [-0.03598446]
 [-0.71139373]
 [ 0.86752805]
 [ 0.33282338]
 [ 0.69522579]
 [ 0.99416438]
 [ 0.40281112]
 [ 0.64809091]
 [-0.50945984]
 [-0.99521012]
 [-0.30860244]
 [-0.30769739]
 [-0.96581919]
 [ 0.67832235]
 [-0.41778035]
 [-0.31582257]
 [ 0.29131261]
 [-0.65613707]
 [-0.23076751]
 [ 0.54201188]
 [-0.65270671]
 [ 0.11449571]
 [ 0.72966007]
 [ 0.61876413]
 [ 0.64107696]
 [-0.96214387]
 [ 0.92316003]
 [ 0.75263269]
 [ 0.6445975 ]
 [-0.25335996]
 [ 0.60436778]
 [-0.01499888]
 [-0.28948127]
 [-0.78335662]
 [ 0.14203317]
 [-0.49525549]
 [-0.66626706]
 [ 0.33282338]
 [ 0.68154845]
 [ 0.98736382]
 [-0.78832829]
 [-0.18001675]
 [ 0.44062007]
 [ 0.03099007]
 [-0.62672237]
 [ 0.00699989]
 [ 0.28121415]
 [ 0.56011449]
 [ 0.42025362]
 [-0.

In [86]:
# BACKPROP
print("======== backprop ========")
# calculate dL/dy : Loss에 대한 y의 gradient
dLdy = output[-1]
dLdy[target_ex] -= 1  # ??
rnn.backprop(dLdy)
print("done....")

======== backprop ========
done....


### 4.3 Training Model

In [133]:
# hyperparameters
output_dim = vocab_size  # embedding_size = 65
hidden_size = vocab_size
epochs = 1
n_batches = n_batches_dataset
# init
hidden_state = np.zeros((hidden_size, ))
max_output = np.zeros((BATCH_SIZE, seq_length, ), int)
losses = list()
# dataset
iterator_dataset = dataset.make_initializable_iterator() # get_next로 하나씩 가져옴
# iterator_dataset = dataset.make_one_shot_iterator()        # 한번에 배치 만들어줌
get_next_dataset = iterator_dataset.get_next()

with tf.Session() as sess:
    
    for e in range(epochs):
        
        sess.run(iterator_dataset.initializer)
        num_correct = 0
        loss = 0.

        for n in range(n_batches):
            
            # load dataset batch
            # input_ex_batch: (n_batches, batch_size, seq_length)
            input_ex_batch, target_ex_batch = sess.run(get_next_dataset)
            
            for i, (input_ex, target_ex) in enumerate(zip(input_ex_batch, target_ex_batch)): 
                
                # tensor는 한 문장 (100, 65, 1)
                input_tensor = idxs2tensor(input_ex)

                rnn = RNNCell(input_dim=input_tensor.shape[1]
                   , output_dim=output_dim
                   , hidden_size=hidden_size
                   , initial_state=hidden_state)
                
                # FORWARD
                outputs, states = rnn.forward(input_tensor)
                # (100, 65, 1) 중에 가장 가능성 높은 것만 선택 -> (100, )
                # argmax default(flatten), axis 0(행), axis 1 (열)
                max_output[i] = [np.argmax(item, axis=0)[0] for item in outputs]

                # loss & accuracy -- working
                # E(y_true, y_pred) = -y_true * log(y_pred)  
                #                   => -sum( E (y_true, y_pred) )
                loss -= sum(np.log([out[target_ex[j]][0] for j, out in enumerate(outputs)]))
                num_correct += np.sum(max_output[i] == target_ex)

                # BACKPROP
                # calculate dL/dy : Loss에 대한 y의 gradient
                dLdy = outputs[-1]
                dLdy[target_ex] -= 1  # ??
                rnn.backprop(dLdy)

        print("EPOCH %d" % e)
        print("loss %.3f" % loss)
        print("accuracy %.3f" % (num_correct / (n_batches * BATCH_SIZE * seq_length)))
        print()
        
# EPOCH 0
# loss 19690762.404
# accuracy 0.016

EPOCH 0
loss 19691004.338
accuracy 0.016



In [134]:
outputs

array([[[ 2.52047388e-06],
        [ 1.45503219e-03],
        [ 1.71837298e-04],
        ...,
        [ 1.01848385e-07],
        [ 8.13332134e-06],
        [ 1.21481563e-07]],

       [[ 7.18196257e-05],
        [ 5.83561463e-13],
        [ 2.74620904e-03],
        ...,
        [ 1.43632921e-05],
        [ 3.27798813e-05],
        [ 1.42437732e-07]],

       [[ 6.37313228e-02],
        [ 1.64679175e-06],
        [ 1.89783994e-04],
        ...,
        [ 2.27590685e-08],
        [ 2.94270644e-06],
        [ 3.40344200e-05]],

       ...,

       [[ 2.47738328e-07],
        [ 6.37796887e-09],
        [ 4.26020964e-07],
        ...,
        [ 1.51191907e-07],
        [ 5.85377641e-05],
        [ 2.47564358e-07]],

       [[ 2.50866306e-09],
        [ 4.31849267e-07],
        [ 9.47055845e-17],
        ...,
        [ 2.11898965e-02],
        [ 1.76976509e-14],
        [ 1.17961334e-07]],

       [[-9.99961861e-01],
        [-9.99999733e-01],
        [ 2.41355721e-03],
        ...,
        

In [137]:
ans = ""
for item in max_output[1]:
    ans += idx2char[item]
ans

".&kt&rIUXFfqdUaXAjAi.eGWd'&baPr?l &3q-kPgvJsRnsSWSq?.A!bowS JISSA$RFoiLm&SwkaBJABTeq?klGnGW.hq-ffwin"

번외 테스트

In [213]:
# hyperparameters
output_dim = 4  # embedding_size = 65
hidden_size = 3
epochs = 1
# init
hidden_state = np.zeros((hidden_size, 1))
# max_output = np.zeros((BATCH_SIZE, seq_length, ), int)
losses = list()

with tf.Session() as sess:
    
    # 5x4 data
    input_test = np.array(
            [[1,2,3,4]
             , [4,5,6,7]
             , [8,9,10,11]
             , [23,45,66,88]
             , [100,102,103,104]])
#     input_test = np.expand_dims(input_test, 1)
#     print(input_test.shape)
    
    rnn = RNNCell(input_dim=input_test.shape[1]
                   , output_dim=output_dim
                   , hidden_size=hidden_size
                   , initial_state=hidden_state)
                
    # FORWARD
    outputs, states = rnn.forward(input_test)
    # (100, 65, 1) 중에 가장 가능성 높은 것만 선택 -> (100, )
    # argmax default(flatten), axis 0(행), axis 1 (열)
#     max_output[i] = [np.argmax(item, axis=0)[0] for item in outputs]

    # loss & accuracy -- working
    # E(y_true, y_pred) = -y_true * log(y_pred)  
    #                   => -sum( E (y_true, y_pred) )
#     loss -= sum(np.log([out[target_ex[j]][0] for j, out in enumerate(outputs)]))
#     num_correct += np.sum(max_output[i] == target_ex)

    # BACKPROP
    # calculate dL/dy : Loss에 대한 y의 gradient
#     dLdy = outputs[-1]
#     dLdy[target_ex] -= 1  # ??
#     rnn.backprop(dLdy)

#     print("loss %.3f" % loss)
#     print("accuracy %.3f" % (num_correct / (n_batches * BATCH_SIZE * seq_length)))
#     print()

In [215]:
outputs

array([[[0.0098708 ],
        [0.10017508],
        [0.2239153 ],
        [0.66603882]],

       [[0.01385864],
        [0.09240407],
        [0.23055724],
        [0.66318004]],

       [[0.00894336],
        [0.10246301],
        [0.22180683],
        [0.6667868 ]],

       [[0.00882494],
        [0.10277904],
        [0.22153518],
        [0.66686085]],

       [[0.00882494],
        [0.10277904],
        [0.22153518],
        [0.66686085]]])

## C. TensorFlow를 이용한 모델